In [3]:
import tomotopy as tp
import spacy
from spacy.tokens import DocBin, Doc
Doc.set_extension("ID", default='')
Doc.set_extension("headline", default='')
Doc.set_extension("label", default='')
import pandas as pd 
import os 
from tqdm import tqdm
import numpy as np
import pandas as pd
import regex as re
re.DEFAULT_VERSION = re.VERSION1
import plotly.express as px
from IPython.display import Image
from gensim.models.phrases import Phraser, Phrases
import time
import gc
from bertopic import BERTopic 
from sklearn.feature_extraction.text import CountVectorizer
from sentence_transformers import SentenceTransformer
import torch
from transformers import AutoTokenizer, AutoModel
import zipfile
from unidecode import unidecode


## Apply NLP pipeline for tokenization, lemmatization and other features for latter uses

In [38]:
textDf = pd.read_csv('data/le_figaro.csv')
textDf = textDf.fillna('')
textDf

,date,pageNum,paragraphNum,content,headline,label
0,1870-01-01,0,9,Admbûstrateur . AUGUSTE DUNIONT ABOMMBHXins Pa...,,
1,1870-01-01,0,13,m part ri pas demain à l'occasion du jour de l...,Le Figaro,
2,1870-01-01,0,20-21-22-23-24,"Notre éclectisme en roli tique qui, pour être ...",,
3,1870-01-01,0,25-26,"fœil bravement fixé sur le couteau, gravi les ...",,
4,1870-01-01,0,27-28,". Hier matii , lés gens dé Batignolles considé...",,
...,...,...,...,...,...,...
791868,1910-12-31,5,7,Le sujet imaginé par MM. Gheusi et Mé rane aur...,LA SOIRÉE LE MIRACLE A L'OPÉRA,culture
791869,1910-12-31,5,9-10-11-12-13-14-15,M lle Chenal M. Muratore si l'on en juge parla...,A L'OPÉRA Le Miracle,culture
791870,1910-12-31,5,16,à 8 heures pour les représentations de M. Gili...,"Ce-soir : A l'Opéra,",culture
791871,1910-12-31,5,18-19,l'Habitation forcée SUITE derrière le vert ble...,Feuilleton du FIGARO du 31 Décembre,autres


In [15]:
nlp = spacy.load('fr_core_news_lg', exclude=["ner"])

In [8]:
doc_bin = DocBin(store_user_data=True)


texts = ((row.content, ('_'.join([str(row['date']),str(row['pageNum']),str(row['paragraphNum'])]), 
                        row['headline'], row['label']))
         for _, row in textDf.iterrows())
count = 0
nDocs = len(textDf)
with tqdm(total=len(textDf),mininterval = 5, miniters =1000) as pbar:
    for doc, (ID, headline, label) in nlp.pipe(texts,as_tuples = True, batch_size=2048,n_process=16):
        doc._.ID = ID
        doc._.headline = headline
        doc._.label = label
        doc_bin.add(doc)
        pbar.update(1)
        #split data into several docbin
        if (count%50_000 == 0) or (count == nDocs-1):
            i = int(count/50_000)-4 if (count != nDocs-1) else int(count/50_000) -3
            if count>0:
                doc_bin.to_disk(f"data/spacy/le_figaro{i}.spacy")
                del doc_bin
                gc.collect()
            doc_bin = DocBin(store_user_data=True)
            
        count+=1


 49%|███████████████▎               | 391872/791873 [1:42:38<1:44:46, 63.63it/s]


In [9]:
len(doc_bin)

0

# Pachinko allocation

In [3]:
nlp = spacy.load('fr_core_news_lg', disable=["ner"])


doc_bin = DocBin(store_user_data=True)
for root, dirs, files in os.walk('data/spacy'):
    for name in tqdm(files):
        if name.endswith((".spacy")):
            doc_bin.merge(DocBin(store_user_data=True).from_disk("./data/spacy/"+name))


100%|███████████████████████████████████████████| 12/12 [01:39<00:00,  8.32s/it]
0it [00:00, ?it/s]


In [47]:
def preprocess_text(doc):
    ID = doc._.ID
    headline = doc._.headline
    label = doc._.label
    oovRatio = np.mean([1 if word.is_oov else 0 for word in doc if word.is_alpha])
    lemmas = [word.lemma_.lower() for word in doc 
           if word.is_alpha and (not word.is_stop) and (len(word.lemma_)>2) and (not word.is_oov)] 
    
    return ID, doc.text, lemmas, oovRatio, headline, label

In [55]:
docs = [preprocess_text(doc) for doc in doc_bin.get_docs(nlp.vocab)]
textDf = pd.DataFrame(docs, columns=['ID','content','lemmatized','oovRatio','headline','label'])
textDf = textDf[(textDf.lemmatized.apply(len)>5) &
                (textDf.oovRatio<0.6) &
                (textDf.label=='')]
print(len(textDf))

709265


In [65]:
raw_articles = textDf["lemmatized"].tolist()
bigram = Phrases(raw_articles, min_count=10, threshold=10)
bigram_mod = Phraser(bigram)
raw_articles = list(bigram_mod[raw_articles])
trigram = Phrases(raw_articles, min_count=10, threshold=10)
trigram_mod = Phraser(bigram)
raw_articles = list(trigram_mod[raw_articles])
textDf["nGram"] = raw_articles
textDf["nGram"] = textDf["nGram"].apply(' '.join)
textDf["lemmatized"] = textDf["lemmatized"].apply(' '.join)
textDf.to_csv('data/le_figaro_lemmatized_without_stop.csv',index=False)

In [ ]:
textDf = pd.read_csv('data/le_figaro_lemmatized_without_stop.csv')
raw_articles = textDf.nGram.str.split().to_list()

In [66]:
corpus = tp.utils.Corpus()
for doc in raw_articles:
    if doc:
        corpus.add_doc(doc)

In [ ]:
k1_max = 7
k1_min = 2 
k2_max = 18
k2_min = 8
params = []
scores = []
num_iter = 0
max_iter = 0
for k1 in range(k1_min, k1_max):
    for k2 in range(max(k1,k2_min),k2_max):
        max_iter +=1
        
start = time.time()
for k1 in range(k1_min, k1_max):
    for k2 in range(max(k1,k2_min),k2_max):
        num_iter+=1
        model = tp.PAModel(tw=tp.TermWeight.IDF,min_cf = 10, min_df=5, rm_top=25, k1=k1, k2=k2, corpus=corpus)
        model.burn_in = 20
        model.train(40, workers=24)
        score = tp.coherence.Coherence(model, coherence="c_v").get_score()
        params.append((k1,k2))
        scores.append(score)
        print("Runtime: %.2f seconds" %(time.time() - start), "|| Number of Searches: %s out of  %s" %(num_iter, max_iter), "|| k1: %s & k2: %s || coherence : %.3f" %(k1,k2,score))

In [82]:
for ind,score in enumerate(scores):
    if score>max(scores)*0.95:
        print("k1: %s & k2: %s || coherence : %.3f" %(params[ind][0],params[ind][1],score))


k1: 3 & k2: 9 || coherence : 0.659
k1: 4 & k2: 11 || coherence : 0.685
k1: 4 & k2: 14 || coherence : 0.694
k1: 5 & k2: 15 || coherence : 0.668


In [91]:
k1 = 4
k2 = 14
model = tp.PAModel(tw=tp.TermWeight.IDF,min_cf = 10, min_df=5, rm_top=25, k1=k1, k2=k2, corpus=corpus)
model.burn_in = 20
model.train(100, workers=24)
score = tp.coherence.Coherence(model, coherence="c_v").get_score()
print(score)

0.7261144071817398


In [94]:
for k in range(k2):
    print('Topic #{}'.format(k))
    print("\t", [w for w, p in model.get_topic_words(k)])

Topic #0
	 ['gouvernement', 'angleterre', 'anglais', 'allemagne', 'ministre', 'russie', 'france', 'français', 'allemand', 'empereur']
Topic #1
	 ['officier', 'armée', 'troupe', 'soldat', 'capitaine', 'militaire', 'navire', 'commandant', 'guerre', 'colonel']
Topic #2
	 ['comte', 'mlle', 'comtesse', 'comte_comtesse', 'marquis', 'baron', 'vicomte', 'baronne', 'paul', 'docteur']
Topic #3
	 ['politique', 'france', 'pays', 'république', 'peuple', 'chose', 'esprit', 'gouvernement', 'estper', 'idée']
Topic #4
	 ['monsieur', 'femme', 'aller', 'oeil', 'dire', 'rien', 'main', 'cœur', 'vie', 'jamais']
Topic #5
	 ['rue', 'maison', 'eau', 'vendre', 'prix', 'franc', 'blanc', 'vin', 'noir', 'gramme']
Topic #6
	 ['prix', 'roi', 'course', 'fête', 'prince', 'cheval', 'palais', 'voiture', 'empereur', 'majesté']
Topic #7
	 ['théâtre', 'représentation', 'pièce', 'artiste', 'opéra', 'jouer', 'mlle', 'soir', 'succès', 'rôle']
Topic #8
	 ['franc', 'chambre', 'commission', 'député', 'loi', 'séance', 'million', 

# BERTopic

In [13]:
class DocSplitor:
    def __init__(self, model = None, tokenizer = None, lim = 382, device='cuda:1'):
        self.device = device
        self.model = model.to(device)
        print(f'Computation on {model.device}')
        self.tokenizer = tokenizer
        self.lim = lim
    
    def mergeList(self, l1,l2):
        return l1+l2
    
    def mergeStr(self, s1,s2):
        return s1+' '+s2
    
    def merge(self, toks1, toks2):
        return {'content':self.mergeStr(toks1['content'],toks2['content']), 
                'tokens':self.mergeList(toks1['tokens'],toks2['tokens']), 
                'lemmas':self.mergeList(toks1['lemmas'],toks2['lemmas']),
                'cndVerb': toks1['cndVerb'] + toks2['cndVerb'],
                'futVerb': toks1['futVerb'] + toks2['futVerb'],
                'nTokens':toks1['nTokens']+toks2['nTokens']}
    
    def mergeAll(self, tokenized):
        if len(tokenized)>2:
            return self.merge(tokenized[0],self.mergeAll(tokenized[1:]))
        else:
            return self.merge(*tokenized) if len(tokenized)>1 else tokenized[0]
    
    def checkLim(self, toks1,toks2):
        return ((toks1['nTokens'] + toks2['nTokens']) <= self.lim)
    
    def resize(self, sentence): 
        tokens = self.tokenizer.encode(sentence.text)[1:-1]
        length = len(tokens)
        ratio = len(tokens)/self.lim
        if (ratio<=1): 
            segments = [(sentence,tokens)]
        else:
            segmentSize = int(length/(int(ratio)+2))+1
            segments = [sentence[segmentSize*i:segmentSize*(i+1)] for i in range((int(ratio)+1))]
            segments  =[(segment,self.tokenizer.encode(segment.text)[1:-1])
                        for segment in segments]
                         
        
        return segments
    
    def sentenceProcessing(self, sentence, tokens):
        lemmas = [word.lemma_.lower() for word in sentence 
                  if word.is_alpha and (not word.is_stop) and (len(word.lemma_)>2) and (not word.is_oov)]
        verbs = [tok for tok in sentence if tok.tag_ == 'VERB']
        cndVerb,futVerb = zip(*[('Cnd' in tok.morph.get('Mood'),'Fut' in tok.morph.get('Tense')) 
                                for tok in verbs]) if len(verbs)>0 else ([],[])
        return {'content': sentence.text,
                'tokens': tokens,
                'lemmas': lemmas,
                'cndVerb':sum(cndVerb),
                'futVerb':sum(futVerb),
                'nTokens': len(tokens)}
    
    def groupSentences(self, tokens, length):
        # function to merge list such that it's stay smaller than lim and each block is approximatly the same size 
        if (length>=3):
            if (tokens[0]['nTokens'] <= tokens[2]['nTokens']) and self.checkLim(tokens[0],tokens[1]):
                return ([self.merge(tokens[0],tokens[1])] + 
                        self.groupSentences(tokens[2:],length-2 ))
            
            elif self.checkLim(tokens[1],tokens[2]):
                return ([tokens[0], self.merge(tokens[1],tokens[2])] + 
                        self.groupSentences(tokens[3:],length-3 ))
            
            else: 
                return (tokens[:2] +
                        self.groupSentences(tokens[2:],length-2 ))
            
        elif (length==2):
            if self.checkLim(tokens[0],tokens[1]):
                return [self.merge(tokens[0],tokens[1])] 
            
            else:
                return tokens
            
        else:
            return tokens

    def tokenize(self, doc):

        if len(doc._.label) != 0 or (len(doc)<50):
            return ('','',[], [], [], [], [], [])
        # [1:-1] to remove delimiter
        
        tokenized = [self.sentenceProcessing(segment,tokens) 
                     for sentence in doc.sents
                     for (segment,tokens) in self.resize(sentence)]
        if sum([sentence['nTokens'] for sentence in tokenized])>self.lim:
            
            mergeable = True
            while mergeable:
                nBlock = len(tokenized)
                tokenized = self.groupSentences(tokenized,nBlock)
                mergeable = not (nBlock == len(tokenized))
        else: 
            tokenized = [self.mergeAll(tokenized)]

        return (doc._.ID, 
                doc._.headline, 
                [str(i) for i in range(len(tokenized))],
                [sentence['content'] for sentence in tokenized], 
                [sentence['lemmas'] for sentence in tokenized], 
                [sentence['cndVerb'] for sentence in tokenized],
                [sentence['futVerb'] for sentence in tokenized],
                [[2]+sentence['tokens']+[3] for sentence in tokenized])
    
    def getAttentionMask(self, attentionLen): 
        return [1]*attentionLen + [0]*(self.lim+2-attentionLen)
    
    
    def batchEmbeds(self, input_ids,attention_mask, tokEmb = False, nToks = None):
        
        output = self.model(input_ids=input_ids.to(self.device),
                            attention_mask=attention_mask.to(self.device),
                            output_hidden_states=tokEmb,
                            output_attentions=False)
        if tokEmb:
            tEmbs = sum(output.hidden_states[-4:])/4
            tEmbs = torch.vstack([tEmbs[ind,:attentionLen,:] 
                                  for ind, attentionLen in enumerate(nToks)])
            return tEmbs.cpu().numpy().astype('float16')
        
        else:
            return output.pooler_output.cpu().numpy().astype('float16')
    
    def saveNumpyAsZip(self, embs,index,path,nbatch):
        
        if os.path.isfile(path):
            with zipfile.ZipFile(path, 'a') as archive:
                with archive.open(f'index{nbatch}.npy','w') as file:
                    np.save(file, index)
                with archive.open(f'embs{nbatch}.npy','w') as file:
                    np.save(file, embs)
        else: 
            with open(path, 'wb') as file:
                np.savez_compressed(file,**{ f'index{nbatch}' :index , f'embs{nbatch}' : embs})
                            
    def getEmbeds(self, tokens, ID, tokEmb = False):
        nToks= [len(toks) for toks in tokens]
        attention_mask = torch.ByteTensor([self.getAttentionMask(attentionLen)
                                          for attentionLen in nToks],device='cpu')
        
        input_ids = torch.nn.utils.rnn.pad_sequence([torch.IntTensor(toks,device='cpu') 
                                                     for toks in [[0]*(self.lim+2)]+tokens], 
                                                     batch_first=True)[1:,:]

        if tokEmb:
            self.getTokEmbeds(input_ids, attention_mask, ID, nToks, batchSize=256)
        else:
            self.getDocEmbeds(input_ids, attention_mask, ID, batchSize=512)
        
    def getDocEmbeds(self, input_ids, attention_mask, ID, saveFreq =100, batchSize = 512):
        docEmbs = []
        count = 0
        with torch.no_grad():
            for batch in tqdm(range(0,len(ID),batchSize)):
                
                dEmbs = self.batchEmbeds(input_ids[batch:batch+batchSize,:],
                                         attention_mask[batch:batch+batchSize,:])
                
                docEmbs.append(dEmbs)
                
                if ((batch%(batchSize*saveFreq)== 0) and(batch>0))  or (batch+batchSize>=len(ID)):
                    
                    docInd = np.array(ID[batchSize*saveFreq*count:batchSize*saveFreq*(count+1)])
                    self.saveNumpyAsZip(np.vstack(docEmbs),docInd,'data/bertEmbeddings/docEmbs.npz',count)
                    del docEmbs, docInd
                    docEmbs = []
                    torch.cuda.empty_cache()
                    gc.collect()
                    count+=1
                                    
    def getTokEmbeds(self, input_ids, attention_mask, ID, nToks, saveFreq =1000, batchSize = 256):
        tokEmbs =  []
        count = 0
        
        with torch.no_grad():
            for batch in tqdm(range(0,len(ID),batchSize)):
                tEmbs = self.batchEmbeds(input_ids[batch:batch+batchSize,:],
                                         attention_mask[batch:batch+batchSize,:],
                                         tokEmb = True,
                                         nToks = nToks[batch:batch+batchSize])
                
                tokEmbs.append(tEmbs)
                
                if ((batch%(batchSize*saveFreq)== 0) and(batch>0))  or (batch+batchSize>=len(ID)):
                    
                    tokInd = np.array([(ID[batchSize*saveFreq*count+i],tokIndex) 
                                       for i, attentionLen 
                                       in enumerate(nToks[batchSize*saveFreq*count:batchSize*saveFreq*(count+1)])
                                       for tokIndex in range(attentionLen)])
                    #self.tokInd = tokInd
                    #self.tokEmbs = tokEmbs
                    self.saveNumpyAsZip(np.vstack(tokEmbs),tokInd,'data/bertEmbeddings/tokEmbs.npz',count)
                    del tokEmbs, tokInd
                    tokEmbs =  []
                    torch.cuda.empty_cache()
                    gc.collect()
                    count+=1
                    

In [5]:
tokenizer = AutoTokenizer.from_pretrained("dbmdz/bert-base-french-europeana-cased")
model =  AutoModel.from_pretrained('dbmdz/bert-base-french-europeana-cased')

Some weights of the model checkpoint at dbmdz/bert-base-french-europeana-cased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [22]:
BERTDf

,ID,headline,segmentID,content,lemmas,cndVerb,futVerb,tokens
0,1898-03-21_0_27_0,De notre correspondant de,0,Budapest : A l'occasion du prochain jubilé du ...,"[budapest, occasion, prochain, jubilé, règne, ...",0,5,"[2, 16466, 30, 37, 79, 11, 2213, 378, 2086, 71..."
2,1898-03-21_0_29_0,,0,Au Tribunal correctionnel : X Le président . E...,"[tribunal, correctionnel, président, avoir, vo...",0,0,"[2, 748, 11746, 13330, 30, 60, 447, 1031, 18, ..."
3,1898-03-21_0_30_0,,0,. Nous aurons le plaisir d'offrir demain à nos...,"[avoir, plaisir, offrir, demain, lecteur, prim...",0,1,"[2, 18, 970, 10172, 354, 3906, 71, 11, 7002, 2..."
4,1898-03-21_0_31_0,POUR ALICE LA VIGNE AVEUGLE -,0,Nous ne publierons que d tnain la quatrième li...,"[publier, liste, souscription, recevoir, repré...",0,0,"[2, 970, 446, 7212, 406, 393, 71, 6477, 468, 3..."
5,1898-03-21_0_32-33_0,De notre correspondant de,0,"Nice : t Ce matin, deux services ont-été céléb...","[nice, matin, service, être, célébrer, neuf, h...",1,0,"[2, 5759, 30, 87, 860, 974, 16, 603, 3186, 507..."
...,...,...,...,...,...,...,...,...
791865,1878-05-13_2_9-10-11-12-13-14-15_11,PREMIÈRES KEFEÉSEHTATIONS,11,-soir dimanche à la représentation clés Sept C...,"[soir, dimanche, représentation, clé, château,...",0,1,"[2, 17, 897, 1993, 130, 348, 4079, 25010, 1242..."
791866,1878-05-13_2_17-18_0,,0,coursés au bois de boulogne Les journées se su...,"[bois, boulogne, journée, longchamps, brillant...",0,0,"[2, 1009, 381, 366, 2590, 336, 5502, 2293, 519..."
791866,1878-05-13_2_17-18_1,,1,Ce costume est très à-la mode. Il m'a paru aus...,"[costume, mode, paraître, chapeau, fort, genre...",1,0,"[2, 860, 10300, 401, 739, 130, 17, 348, 4726, ..."
791866,1878-05-13_2_17-18_2,,2,Mais nous ne manquons pas de courses en semain...,"[manquer, course, semaine, pouvoir, rattraper,...",0,5,"[2, 811, 512, 446, 774, 19943, 435, 336, 4843,..."


In [20]:
#splitor = DocSplitor(model,tokenizer)
            
#docs = [splitor.tokenize(doc) for doc in doc_bin.get_docs(nlp.vocab)]
#BERTDf = pd.DataFrame(docs, columns=['ID','headline','segmentID','content','lemmas','cndVerb','futVerb','tokens'])
BERTDf = BERTDf.explode(['segmentID','content','cndVerb','futVerb','lemmas','tokens'])
BERTDf  =BERTDf[~BERTDf.content.isna()].copy()
BERTDf['ID'] += '_' +BERTDf['segmentID'] 
BERTDf.to_pickle('data/le_figaro_BERT.bz',)

del docs
del doc_bin
gc.collect()

96

In [4]:
BERTDf = pd.read_pickle('data/le_figaro_BERT.bz',)


#### Save doc embeddings

In [298]:
splitor.getEmbeds(BERTDf.tokens.tolist(),BERTDf.ID.tolist())
torch.cuda.empty_cache()
gc.collect()

Computation on cuda:0


100%|████████████████████████████████████| 2675/2675 [11:00:31<00:00, 14.82s/it]


#### Save token embeddings for filtered corpus

In [302]:
topic = {'science':['découverte','science','scienti','techni'],
         'futur': ['déclin','décadence','futur','avenir','progrès']}
keywords = topic['science'] + topic['futur']

BERTDfKws = BERTDf[BERTDf.content.apply(unidecode).str.lower().apply(lambda x : any(kw in x for kw in keywords))]

Computation on cuda:0


In [427]:
BERTDfKws.to_pickle('data/le_figaro_BERT_filtered.gz')

In [426]:
splitor.getEmbeds(BERTDfKws.tokens.tolist(),BERTDfKws.ID.tolist(), tokEmb=True)

Computation on cuda:1


100%|███████████████████████████████████████| 352/352 [1:11:25<00:00, 12.17s/it]


In [6]:
with open(f'data/bertEmbeddings/docEmbs.npz', 'rb') as tokFile:
    zipedFiles = np.load(tokFile)
    embs = [zipedFiles[file][:len(zipedFiles[file])]
            for file in zipedFiles if 'embs' in file]
    index = [zipedFiles[file][:len(zipedFiles[file])]
            for file in zipedFiles if 'index' in file]
    docEmbs = pd.DataFrame(np.vstack(embs),index =np.concatenate(index)).apply(np.array,axis=1)
    del embs
    del index

In [7]:
#del splitor
torch.cuda.empty_cache()
gc.collect()

461

In [24]:

topic_model = BERTopic(verbose=True, min_topic_size = 40, language=None)
BERTDf = BERTDf.set_index('ID').loc[docEmbs.index]


In [9]:

topics, probs = topic_model.fit_transform(BERTDf.content.tolist(),
                                          np.array(docEmbs.tolist()))
topic_model.save("BERTopic.bin",save_embedding_model=False)

2023-01-23 06:15:53,308 - BERTopic - Reduced dimensionality
2023-01-23 06:24:56,678 - BERTopic - Clustered reduced embeddings


In [125]:
topic_model = BERTopic().load("BERTopic.bin")

In [ ]:
topics_over_time = topic_model.topics_over_time(BERTDf.content.tolist(), 
                                                [ID.split('_')[0] for ID in docEmbs.index],
                                                nr_bins = 20,
                                                global_tuning=False)

In [ ]:
topic_model.visualize_topics_over_time(topics_over_time,normalize_frequency=False,top_n_topics=20)

In [126]:
vectorizer_model = CountVectorizer(ngram_range=(1,3), 
                                   max_df = 0.6,
                                   min_df = 25,
                                   #tokenizer= tokenizer,
                                   #strip_accents = 'unicode',
                                   lowercase = True,
                                   stop_words = nlp.Defaults.stop_words)
#topic_model.update_topics(BERTDf.content.tolist(), vectorizer_model = vectorizer_model)
vectorizer_model = vectorizer_model.fit(BERTDf.content.tolist())

In [ ]:
topics_over_time = topic_model.topics_over_time(BERTDf.content.tolist(), 
                                                [ID.split('_')[0] for ID in docEmbs.index],
                                                nr_bins = 20,
                                                global_tuning=True)

In [35]:
topic_model.visualize_topics_over_time(topics_over_time,normalize_frequency=True,top_n_topics=20)

In [127]:
checkAny = lambda ngrams, cond : any(cond(word) for word in ngrams.split())
checkAll = lambda ngrams, cond : all(cond(word) for word in ngrams.split())
checkLen = lambda word : (len(word)>2)
checkIsAlpha = lambda word : word.isalpha()
checkIsValid = lambda word: word in nlp.vocab.strings and word not in nlp.Defaults.stop_words

vocab = {unidecode(words) for words in vectorizer_model.vocabulary_ 
         if  checkAny(words,checkLen) and checkAll(words,checkIsAlpha) and checkAny(words,checkIsValid)}


In [136]:
vectorizer_model = CountVectorizer(vocabulary = vocab)
topic_model.update_topics(BERTDf.content.tolist(), vectorizer_model = vectorizer_model)


/home/gsicard/.local/lib/python3.9/site-packages/bertopic/vectorizers/_ctfidf.py:69: RuntimeWarning:

divide by zero encountered in true_divide



In [3]:
import numpy as np
np.array([1,2,3],dtype='float32')

array([1., 2., 3.], dtype=float32)

In [134]:
topics_over_time = topic_model.topics_over_time(BERTDf.content.tolist(), 
                                                [ID.split('_')[0] for ID in docEmbs.index],
                                                nr_bins = 20,
                                                global_tuning=True)

0it [00:35, ?it/s]


ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

In [124]:
topic_model.get_topic_info()

,Topic,Count,Name
0,-1,873297,-1_ete_bien_hier_paris
1,0,81812,0_yeux_fit_oui_femme
2,1,41772,1_bien_france_faire_politique
3,2,40586,2_bien_faut_faire_france
4,3,23766,3_gouvernement_berlin_troupes_depeche
...,...,...,...
439,438,40,438_chapeau_noir_chapeau noir_plumes
440,439,40,439_sympathie_telegramme_condoleances_douleur
441,440,40,440_liberaux_gouvernement_nationale_republicains
442,441,40,441_hotel_beau jardin hotel_trouve rez_retenir...


In [128]:
len(vectorizer_model.vocabulary_)

535056

In [129]:
len(vocab)

479481

In [88]:
'aspirations' in nlp.vocab.strings

True

In [130]:
vocab

{'filieres',
 'charles ligne',
 'rien perdu attendre',
 'mois vient',
 'travers voie',
 'mosa',
 'partie correspondance',
 'aspirations',
 'noyon',
 'marquis marquise san',
 'mont ferrand',
 'soir mm',
 'citoyens honorables',
 'circonstances actuelles',
 'guerre venue',
 'succes derniers',
 'annuelle reouverture',
 'usage immemorial',
 'pimprenelle',
 'location',
 'laissant',
 'gymnastes excentricites',
 'documentaire',
 'demission bien',
 'titre union',
 'docteur bouloumie',
 'etage',
 'verrerie ouvriere',
 'faits ailleurs',
 'simple digne',
 'monda',
 'mauvaise digestion',
 'ete com',
 'habitaient maison',
 'robert lubez',
 'lecteurs renomme',
 'naturellement points',
 'habit porte',
 'fer ailleurs',
 'laisse champ',
 'avis bon',
 'royaliste roi',
 'vacance siege',
 'calque',
 'theatre adrien bernheim',
 'voulu prevenir',
 'nice toulon',
 'registres abonnements',
 'forte odeur',
 'panier qu',
 'releve parla',
 'inspecteur principal gardiens',
 'debarrassez',
 'service telegraphique f

In [132]:
vectorizer_model.vocabulary_

{'budapest': 72035,
 'occasion': 335352,
 'prochain': 384993,
 'jubilé': 263384,
 'règne': 436002,
 'françois': 204265,
 'joseph': 259164,
 'autrichiens': 49380,
 'hongrois': 236066,
 'élevés': 523286,
 'rang': 413515,
 'prince': 381336,
 'barons': 54687,
 'recevront': 416039,
 'titre': 482271,
 'comte': 101271,
 'grand': 216738,
 'nombre': 327142,
 'fonctionnaires': 195482,
 'civils': 92452,
 'militaires': 302947,
 'créés': 121804,
 'intéressante': 251206,
 'particularité': 353787,
 'signaler': 454705,
 'propos': 388733,
 'nouveaux': 330388,
 'titulaires': 482614,
 'blason': 64191,
 'anoblis': 30014,
 'sement': 451718,
 'constitue': 109145,
 'crois': 120310,
 'précédent': 391015,
 'europe': 174183,
 'appartiennent': 32777,
 'religion': 418826,
 'maho': 285189,
 'efa': 161043,
 'bosniaques': 67483,
 'bourgmestre': 68673,
 'serajevo': 452863,
 'deviendra': 132348,
 'mohamed': 311275,
 'beg': 57380,
 'reis': 418334,
 'el': 161784,
 'chef': 86650,
 'magistrature': 284948,
 'religieuse': 4